In [2]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Data collection and preprocessing

In [4]:
movies_data = pd.read_csv(r"C:\Users\ayanm\Desktop\Data for ML\movies.csv")

In [5]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
movies_data.shape

(4803, 24)

In [7]:
# selecting relevant features for recommendation
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [8]:
movies_data.genres.isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
4798    False
4799    False
4800    False
4801     True
4802    False
Name: genres, Length: 4803, dtype: bool

In [9]:
# replacing the null values with null string
for feature in selected_features :
    movies_data[feature] = movies_data[feature].fillna('')

In [10]:
# combining all the 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [11]:
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

### Transforming raw text into numerical features using TfidVectorizer

In [13]:
# converting text data to feature vectors
vectorizer = TfidfVectorizer()

In [14]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [15]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

## Cosine Similarity

In [17]:
# similarity score using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [18]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [19]:
 similarity.shape

(4803, 4803)

In [20]:
# getting the movie name from user
movie_name = input ('Enter your favourite movie name: ')

Enter your favourite movie name:  Captain America


In [21]:
# creating a list with all the movie names given in the dataset
list_of_all_titles = movies_data['title'].tolist()
list_of_all_titles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

### Difflib to find close matches with the input movie by user

In [23]:
# finding the close match for the movie name given by user
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_match

['Captain America: Civil War',
 'Mi America',
 'Captain America: The First Avenger']

In [24]:
close_match = find_close_match[0]
close_match

'Captain America: Civil War'

In [25]:
# finding the index of the movie with title
index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
index_of_the_movie

26

In [26]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [27]:
# sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
sorted_similar_movies

[(26, 0.9999999999999998),
 (85, 0.5992323779418586),
 (7, 0.45873936834252743),
 (16, 0.4046969407600009),
 (79, 0.31353387110940273),
 (169, 0.30943494149856277),
 (31, 0.22176049914086776),
 (68, 0.21566241096831154),
 (126, 0.20995860723957066),
 (511, 0.18555552518616772),
 (182, 0.17498602091215604),
 (2578, 0.17034039288105401),
 (64, 0.16996886110613166),
 (203, 0.16462252305231945),
 (174, 0.1635256436844351),
 (4401, 0.16114873691796516),
 (624, 0.1603771271081481),
 (101, 0.15999012245360686),
 (129, 0.15859616283733802),
 (38, 0.1574031839792272),
 (1240, 0.15097881992651624),
 (628, 0.14891571396578573),
 (1564, 0.14843603379806108),
 (788, 0.14781693106136182),
 (33, 0.145416424125196),
 (663, 0.14158979602269067),
 (800, 0.14115876594729612),
 (782, 0.140636890525321),
 (47, 0.13943169647444587),
 (46, 0.13935140015240058),
 (855, 0.13899266563113916),
 (1187, 0.13888748954018593),
 (870, 0.13753012901051295),
 (2585, 0.13575557523032833),
 (2550, 0.1333585281308044),
 (

In [28]:
# print the name of similar movies based on the index
print('Movies suggested for you :\n')
i = 1
for movie in sorted_similar_movies :
    index = movie[0]
    title_from_index = movies_data[movies_data.index == index]['title'].values[0]
    if (i<11):
        print(i,'.',title_from_index)
        i+=1

Movies suggested for you :

1 . Captain America: Civil War
2 . Captain America: The Winter Soldier
3 . Avengers: Age of Ultron
4 . The Avengers
5 . Iron Man 2
6 . Captain America: The First Avenger
7 . Iron Man 3
8 . Iron Man
9 . Thor: The Dark World
10 . X-Men
